<a href="https://colab.research.google.com/github/dave502/NLP/blob/main/%20lesson_14/nlp_hw_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers

In [ ]:
import json
import re
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments, TextDataset
import torch
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

###Обучение GPT

#####Загрузка данных

In [ ]:
%%capture
!rm archive_bashim.zip
!rm dataset.jsonl
!wget https://www.dropbox.com/s/670ij0wukat9ynz/archive_bashim.zip
!unzip archive_bashim.zip
!rm archive_bashim.zip

In [ ]:
!ls

dataset.jsonl  sample_data


In [ ]:
!head -n 3 dataset.jsonl

{"id":1,"date":"2004-08-30T11:24:00Z","rating":22010,"text":"<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)"}
{"id":2,"date":"2004-08-30T11:25:00Z","rating":25105,"text":"<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/"}
{"id":3,"date":"2004-08-30T11:27:00Z","rating":7192,"text":"<Дор> \"мышка, почему у тебя такие большие глаза?\" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!"}


In [ ]:
quotes = []
with open('dataset.jsonl') as file:
  for line in file:
    quote_dict = json.loads(line)
    quotes.append(quote_dict.get('text'))

print(len(quotes))
print(*quotes[:3]) 

81497
<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;) <томатик_рад> а ты не чувствуешь красоту мира?
<fox> честно говоря, я сейчас чувствую только отсутствие http.
<томатик_рад> не туда смотришь, глянь вокруг!
<fox> как я гляну, если http не работает? :/ <Дор> "мышка, почему у тебя такие большие глаза?" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!


#####Предобаботка данных

In [ ]:
# удаляются только имена юзеров
def preprocessing(text_lines:list, sep:str='')->list:
  """
  функция педобработки данных
  """  
  # регулярное выражение для очистки текста
  #url_pattern = "(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])"
  #smile_pattern = "(:D|D:)"
  usernames_pattern = "(<[\w]*>)"
  #onlywords_pattern = "[^\w\b]|[^A-Za-zА-ЯЁа-яё]"
  pattern = f"{usernames_pattern}" # |{url_pattern}|{smile_pattern}|{onlywords_pattern}

  preprocessed = [re.sub(pattern, ' ', line) for line in text_lines]
  preprocessed = [(re.sub(r'\s+', ' ', line)+sep).lstrip() for line in preprocessed]

  return preprocessed

In [ ]:
sep = '\n***\n'
data = preprocessing(quotes, sep)
print(*data[:3])

ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)
***
 а ты не чувствуешь красоту мира? честно говоря, я сейчас чувствую только отсутствие http. не туда смотришь, глянь вокруг! как я гляну, если http не работает? :/
***
 "мышка, почему у тебя такие большие глаза?" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!
***



In [ ]:
# список цитат - в один текст
prefix_len = 10
prefix = sep + ''.join(random.sample(data, prefix_len))
print(prefix)


***
Произнеси наоборот "Аргентина манит негра" э... "негр манит аргентину"? о_0
***
Товарищ: знакомая понесла свою кошку в ветеринарку.. думала её операцию сделать чтобы по весне та не хотела.. Товарищ: уже оперировать собрались, а это оказался кастрированный кот О_о
***
B@nshee - Сегодня отправил столько бумажных писем, что чувствую себя спамером-старовером...
***
Работаю в достаточно большой компании ИТшником, большая часть пользователей - девушки и женщины, зоны ответственности очень сильно разграничены и обычно с странными запросами не приходят. Я никогда не рассчитывал на запрос начинающийся со слов: Вы знаете, мы тут на 4 этаж аквариум купили, не могли бы вы нам помочь...
***
apelsinka (15:15:04 11/02/2008) хороший, у нас на проходной то же такой бегает, только попушистей и морда такая добродушная, я ему сосиски ношу, так он меня теперь узнаёт и играет со мной Лёха (15:16:10 11/02/2008) блин лучше бы ты мне сосиски носила я бы с тобой тоже играл........
***
xxx: Знакомые знакомы

Инициализация моделей

In [ ]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

Токенизация текста

In [ ]:
tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k,v in tokens.items()}
eos_token_id = tokenizer.encode(sep)[0]

Генерация текста без дополнительного обучения модели

In [ ]:
length = tokens['input_ids'].shape[1]

output = model.generate(**tokens,
                        eos_token_id=eos_token_id,
                        do_sample=False,
                        max_length=length+64,
                        repetition_penalty=2.,
                        temperature=50,
                        num_beams=1)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:203 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


ustice(141): Нуууу....я просто хочу сказать спасибо за помощь ^__~ *kisses's baggage** kashinenko :))))) jessiehlikova @ mailru   
 *** JAKINOVEYN


#####Дообучение модели

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
training_arguments = TrainingArguments(
    output_dir='gpt2_trainig',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_steps=400,
    save_steps=800,
    warmup_steps=500
    ) 

Запись обучающих и валидационных данных в отдельные файлы и формирования из них датасетов

In [ ]:
train_size = int(len(quotes) * 0.9)
train_file_name = 'train_file'
valid_file_name = 'valid_file'

train_dataset_list = preprocessing(quotes[:train_size], sep)
train_dataset = sep + ''.join(train_dataset_list)
with open(train_file_name, 'w') as train_file:
  train_file.write(train_dataset)
train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_file_name,
          block_size=128)

val_dataset_list = preprocessing(quotes[train_size:], sep)
val_dataset = sep + ''.join(val_dataset_list)
with open(valid_file_name, 'w') as valid_file:
  valid_file.write(val_dataset)
val_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=valid_file_name,
          block_size=128)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Creating features from dataset file at 
Saving features into cached file cached_lm_GPT2TokenizerFast_128_train_file [took 0.178 s]
Creating features from dataset file at 
Saving features into cached file cached_lm_GPT2TokenizerFast_128_valid_file [took 0.018 s]


Дообучение модели

In [ ]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 39496
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 29622
  Number of trainable parameters = 125231616


Step,Training Loss
500,4.033100
1000,4.009900
1500,4.008100
2000,3.983100
2500,3.991100
3000,3.993500
3500,3.959700
4000,3.956700
4500,3.963600
5000,3.962200


Saving model checkpoint to gpt2_trainig/checkpoint-800
Configuration saved in gpt2_trainig/checkpoint-800/config.json
Model weights saved in gpt2_trainig/checkpoint-800/pytorch_model.bin
Saving model checkpoint to gpt2_trainig/checkpoint-1600
Configuration saved in gpt2_trainig/checkpoint-1600/config.json
Model weights saved in gpt2_trainig/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to gpt2_trainig/checkpoint-2400
Configuration saved in gpt2_trainig/checkpoint-2400/config.json
Model weights saved in gpt2_trainig/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to gpt2_trainig/checkpoint-3200
Configuration saved in gpt2_trainig/checkpoint-3200/config.json
Model weights saved in gpt2_trainig/checkpoint-3200/pytorch_model.bin
Saving model checkpoint to gpt2_trainig/checkpoint-4000
Configuration saved in gpt2_trainig/checkpoint-4000/config.json
Model weights saved in gpt2_trainig/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to gpt2_trainig/checkpoint-48

TrainOutput(global_step=29622, training_loss=3.6643122069768537, metrics={'train_runtime': 5640.2647, 'train_samples_per_second': 21.008, 'train_steps_per_second': 5.252, 'total_flos': 7739992571904000.0, 'train_loss': 3.6643122069768537, 'epoch': 3.0})

In [ ]:
trainer.save_model()
model.save_pretrained('gpt_model')
tokenizer.save_pretrained('gpt_tokenizer')

Токенизация и генерация текста дообученной моделью

In [ ]:
tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k,v in tokens.items()}
eos_token_id = tokenizer.encode(sep)[0]

In [ ]:
output = model.generate(**tokens,
                        eos_token_id=eos_token_id,
                        do_sample=False,
                        max_length=length+64,
                        repetition_penalty=2.,
                        temperature=50,
                        num_beams=2)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]

In [ ]:
print(result)

хххвьётся. У меня такое ощущение, что все равно что-то вроде как-то? :) хххх: ну дык там где-то рядом с серверной дверью стоит. В смысле? :(



*Результат вполне удовлетворительный. Немного бессвязно, возможно имело смысл ещё дообучить модель.*

###Обучение T5

In [1]:
%%capture
!pip install transformers[sentencepiece]

In [2]:
# wrapping output
from IPython.display import HTML, display
def set_css():
  display(HTML('''<style>pre{white-space: pre-wrap;}</style>'''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
%%capture
!pip install corus
!pip install rouge
!pip install dataset

In [4]:
from corus import load_lenta
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
import torch
from sklearn.model_selection import train_test_split

#####Загрузка данных

In [5]:
%%capture
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [6]:
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [7]:
NUM_SAMPLES = 20
articles = []
titles = []
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
counter = 0
for record in records:
  articles.append(record.text)
  titles.append(record.title)
  counter += 1
  if NUM_SAMPLES and counter == NUM_SAMPLES: break

In [8]:
articles[:2]

['Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
 'Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо 

In [9]:
titles[:5]

['Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
 'Австрия не\xa0представила доказательств вины российских биатлонистов',
 'Обнаружено самое счастливое место на\xa0планете',
 'В США раскрыли сумму расходов на\xa0расследование «российского дела»',
 'Хакеры рассказали о\xa0планах Великобритании заминировать Севастополь']

#####Разделение датасета на обучающий, тестовый и валидационный. Пока что нам понадобится только тестовый.

In [10]:
train_val_articles, test_articles, train_val_titles, test_titles = train_test_split(articles, titles, test_size=0.2)
train_articles, val_articles, train_titles, val_titles = train_test_split(articles, titles, test_size=0.25)

print(f'{len(train_articles)=}\n{len(train_titles)=}\n***\n{len(val_articles)=} \
      \n{len(val_titles)=}\n***\n{len(test_articles)=}\n{len(test_titles)=}')

len(train_articles)=15
len(train_titles)=15
***
len(val_articles)=5       
len(val_titles)=5
***
len(test_articles)=4
len(test_titles)=4


Инициализация модели

In [16]:
%%capture
model_name = "IlyaGusev/rut5_base_headline_gen_telegram"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

#####Проверим работу модели без дообучения

Токенизация тестовой выборки

In [25]:
input_ids = tokenizer(
    test_articles,
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)["input_ids"]

Генерация заголовков

In [26]:
output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)


Пробразование эмбеддингов в текст

In [27]:
titles = []
for ids in output_ids:
  titles.append(
      tokenizer.decode(
        ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
      )
  )

Полученные заголовки

In [28]:
titles[:5]

['Британцы будут платить семь евро за въезд в ЕС после Brexit',
 'Еврокомиссия пригласила Россию и Украину в Брюссель для переговоров по газу',
 'Судимая в США Бутина может стать фигурантом еще одного дела',
 '«Эвотор» рассказал о стоимости новогоднего стола в России']

Тестовые заголовки

In [29]:
test_titles[:5]

['Британцам придется заплатить за\xa0путешествия по\xa0Европе',
 'Россию и\xa0Украину пригласили на\xa0переговоры по\xa0газу',
 'Бутину допросят в\xa0США по\xa0новому уголовному делу',
 'Названы регионы России с\xa0самым дорогим и\xa0дешевым новогодним столом']

Уже видно, что результат очень хороший, но посмотрим метрики и затем сравним с метриками дообученной модели.

In [30]:
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [31]:
calc_scores(test_titles, titles)

Count: 4
Ref: Названы регионы России с самым дорогим и дешевым новогодним столом
Hyp: «Эвотор» рассказал о стоимости новогоднего стола в России
BLEU:  0.34590326293979023
ROUGE:  {'rouge-1': {'r': 0.2794642857142857, 'p': 0.21761363636363634, 'f': 0.24269005360328832}, 'rouge-2': {'r': 0.14285714285714285, 'p': 0.10277777777777777, 'f': 0.1194852916761057}, 'rouge-l': {'r': 0.2794642857142857, 'p': 0.21761363636363634, 'f': 0.24269005360328832}}


#####Дообучение модели

Токенизация всех датасетов

In [32]:
train_encodings = tokenizer(train_articles, truncation=True, padding=True)
val_encodings = tokenizer(val_articles, truncation=True, padding=True)
test_encodings = tokenizer(test_articles, truncation=True, padding=True)

train_enc_titles = tokenizer(train_titles, truncation=True, padding=True)
val_enc_titles = tokenizer(val_titles, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Класс датасета для Trainer, содержащий статьи и заголовки

In [33]:
class ArticlesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return len(self.labels)

Формируем обучающий и валидационный датасеты

In [34]:
train_dataset = ArticlesDataset(train_encodings, train_enc_titles)
val_dataset = ArticlesDataset(val_encodings, val_enc_titles)

Собственно дообучение модели

In [40]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=500,             # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [41]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)
trainer.train()

***** Running training *****
  Num examples = 2
  Num Epochs = 500
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 244309248


Step,Training Loss
10,8.943300
20,9.235500
30,8.738100
40,8.591200
50,8.254500
60,7.601800
70,7.315800
80,7.136900
90,6.662800
100,5.494100


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=500, training_loss=2.6961772725442423, metrics={'train_runtime': 231.8087, 'train_samples_per_second': 4.314, 'train_steps_per_second': 2.157, 'total_flos': 799221749760000.0, 'train_loss': 2.6961772725442423, 'epoch': 500.0})

Генерация результата дообученной моделью и проверка метрик

In [46]:
input_ids = tokenizer(
    test_articles,
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids.cuda(),
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)

titles = []
for ids in output_ids:
  titles.append(
      tokenizer.decode(
        ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
      )
  )

calc_scores(test_titles, titles)


Count: 4
Ref: Названы регионы России с самым дорогим и дешевым новогодним столом
Hyp: Названы регионы России с самой высокой стоимость новогодних блюд
BLEU:  0.4323148226563139
ROUGE:  {'rouge-1': {'r': 0.3544642857142857, 'p': 0.3173611111111111, 'f': 0.33304093194696494}, 'rouge-2': {'r': 0.2619047619047619, 'p': 0.2326388888888889, 'f': 0.24448529164150362}, 'rouge-l': {'r': 0.3544642857142857, 'p': 0.3173611111111111, 'f': 0.33304093194696494}}


Без дообучения были метрики:
```
BLEU:  0.34590326293979023
ROUGE:  {'rouge-1': {'r': 0.2794642857142857, 'p': 0.21761363636363634, 'f': 0.24269005360328832}, 'rouge-2': {'r': 0.14285714285714285, 'p': 0.10277777777777777, 'f': 0.1194852916761057}, 'rouge-l': {'r': 0.2794642857142857, 'p': 0.21761363636363634, 'f': 0.24269005360328832}}
```
*Метрики BLEU и ROUGE после дообучения стали лучше*


Посмотрим полученные заголовки:

In [47]:
titles[:5]

['Великобритания заплатит семь евро за посещение стран Евросоюза',
 'Источник: ЕК пригласила Россию и Украину на переговоры по газу',
 'СМИ: Россиянка Бутина может стать фигурантом еще одного дела',
 'Названы регионы России с самой высокой стоимость новогодних блюд']

Эти же тестовые заголовки:

In [48]:
test_titles[:5]

['Британцам придется заплатить за\xa0путешествия по\xa0Европе',
 'Россию и\xa0Украину пригласили на\xa0переговоры по\xa0газу',
 'Бутину допросят в\xa0США по\xa0новому уголовному делу',
 'Названы регионы России с\xa0самым дорогим и\xa0дешевым новогодним столом']

*Видно, что некоторые заголовки, несмотря на улучшение метрик, имеют ошибки. Наиболее вероятная причина - модель чрезмерно обучилась, следует снизить количество обучающих эпох.*